In [ ]:
import openpyxl
import json
from typing import Dict, Any, Optional, List, Tuple
from dataclasses import dataclass
from pathlib import Path

@dataclass
class FieldConfig:
    """Configuration for each field to extract"""
    name: str
    positions: List[Tuple[int, int]]  # (row, col) tuples only

def make_field_range(name: str, rows: range, col: int) -> FieldConfig:
    """Create a FieldConfig from a row range and a single column"""
    return FieldConfig(name, [(r, col) for r in rows])
def make_middle( row: int, col: int) -> FieldConfig:
    fields = []
    name=["customer","manufacturer","sevensix"]
    for i in range(3):
        rows = range(row+i*5,row+i*5+5)
        fields.extend([FieldConfig("ai_"+name[i]+"_who", [(r, col) for r in rows]),
        FieldConfig("ai_"+name[i]+"_what", [(r, col+1) for r in rows]),
        FieldConfig("ai_"+name[i]+"_when", [(r, col+5) for r in rows])])
    return fields

def make_case_fields(case_no: int, start_row: int, start_col: int = 6) -> List[FieldConfig]:
    """
    Generate FieldConfigs for a case block.
    case_no: number suffix (1,2,3,...)
    start_row: top row of the case block
    start_col: first column (default=6 for your sheet)
    """
    fields = [
        ("customer_name", (start_row, start_col)),
        ("customer_representative", (start_row, start_col+1)),
        ("occurrence_date", (start_row, start_col+2)),
        ("branch", (start_row, start_col+3)),
        ("application", (start_row+1, start_col)),
        ("manufacturer", (start_row+2, start_col)),
        ("product", (start_row+3, start_col)),
        ("competitive", (start_row+4, start_col)),
        ("amount", (start_row+5, start_col)),
        ("order_month", (start_row+6, start_col)),
        ("budget", (start_row+7, start_col)),
        ("probability", (start_row+8, start_col)),
    ]
    return [FieldConfig(f"case{case_no}_{fname}", [pos]) for fname, pos in fields]

class ExcelDataExtractor:
    """Optimized Excel data extractor (template positions only)"""
    
    def __init__(self):
        self.fields = [
            # Basic Information
            FieldConfig("reporter_name", [(2,4), (2,5), (2,6)]),
            FieldConfig("date", [(3,4), (3,5)]),
            FieldConfig("location", [(4,4), (4,5)]),
            
            # Customer Information
            make_field_range("customer_company_name", range(7, 11), 4),
            make_field_range("customer_department", range(7, 12), 5),
            make_field_range("customer_full_name", range(7, 12), 6),
            FieldConfig("customer_number", [(11,4)]),

            # Manufacturer Information
            make_field_range("manufacturer_company_name", range(7, 11), 7),
            make_field_range("manufacturer_department", range(7, 11), 8),
            make_field_range("manufacturer_full_name", range(7, 11), 9),
            FieldConfig("sevensix", [(7,10),(8,10),(9,10),(10,10)]),
            
            
        ]
        
        # Add multiple case blocks easily
        
        # define columns for middle-table
        
    
    def adjust_fields_by_dimension(self, ws):
        extra_fields = []
        print(ws.max_row)
        if ws.max_row==71:
            self.fields+=[
                # Business Information
            FieldConfig("purpose", [(12,4),(13,4),(16,4)]),
            FieldConfig("free_description", [(17,4),(18,4)]),
            FieldConfig("associated_customer_name", [(35,4)]),
            FieldConfig("competitive_information", [(36,4)]),]
            self.fields.extend(make_middle(20,5))
            self.fields += make_case_fields(case_no=1, start_row=38)
            self.fields += make_case_fields(case_no=2, start_row=48)  # example second case at row 50
            self.fields += make_case_fields(case_no=3, start_row=58)  # third case, etc.
        # check rows
        if ws.max_row == 97:
            self.fields+=[FieldConfig("annual_project_name", [(17,4),(17,5),(17,6)]),
                          FieldConfig("annual_project_schedule", [(18,4),(18,5),(18,6)]),
                          FieldConfig("annual_project_budget", [(19,4),(19,5),(19,6)]),
                          FieldConfig("free_description", [(20,4),(20,5)]),
                          FieldConfig("associated_customer_name", [(46,4)]),
                          FieldConfig("competitive_information", [(47,4)]),
                          ]
            self.fields.extend(make_middle(31,5))

            self.fields += make_case_fields(case_no=1, start_row=49)
            self.fields += make_case_fields(case_no=2, start_row=59)  # example second case at row 50
            self.fields += make_case_fields(case_no=3, start_row=69)  # third case, etc.
            self.fields += make_case_fields(case_no=4, start_row=79)
            self.fields += make_case_fields(case_no=5, start_row=89)



    # ----------- MAIN EXTRACTION ----------
    def extract_from_file(self, file_path: str) -> Dict[str, Any]:
        """Extract values only from predefined positions"""
        try:
            wb = openpyxl.load_workbook(file_path, data_only=True)
            ws = wb.active
            print(f"Processing: {Path(file_path).name} ({ws.title}, {ws.max_row}x{ws.max_column})")
            self.adjust_fields_by_dimension(ws)
            extracted, stats = {}, {"found": 0, "not_found": 0}

            for field in self.fields:
                value = self._extract_field(ws, field)
                extracted[field.name] = value
                stats["found" if value else "not_found"] += 1
            
            results = {
                "data": extracted, 
                "file_info": {
                    "filename": Path(file_path).name,
                    "worksheet": ws.title,
                    "dimensions": f"{ws.max_row}x{ws.max_column}"
                },
                "extraction_stats": {
                    **stats,
                    "total_fields": len(self.fields),
                    "success_rate": f"{(stats['found']/len(self.fields))*100:.1f}%"
                }
            }
            # self._print_summary(results)
            return results
        except Exception as e:
            print(f"❌ Error: {e}")
            return {"error": str(e)}

    # ----------- FIELD EXTRACTION ----------
    def _extract_field(self, ws, field: FieldConfig) -> Optional[str]:
        """Extract value only from predefined positions"""
        values = [
            str(ws.cell(row=r, column=c).value).strip()
            for r, c in field.positions
            if ws.cell(row=r, column=c).value not in (None, "")
        ]
        if values:
            return "\n".join(values) if len(values) > 1 else values[0]
        return None

    # ----------- UTILITIES ----------
    def _print_summary(self, results: Dict[str, Any]):
        if "error" in results:
            print(f"❌ Extraction failed: {results['error']}")
            return
        stats = results["extraction_stats"]
        print(f"\n{'='*50}\nEXTRACTION SUMMARY\n{'='*50}")
        print(f"📊 Success Rate: {stats['success_rate']}")
        print(f"✅ Found: {stats['found']}")
        print(f"❌ Not found: {stats['not_found']}")
        print(f"📋 Total fields: {stats['total_fields']}")

    def save_results(self, results: Dict[str, Any], output_file: str):
        try:
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=2)
            print(f"💾 Saved: {output_file}")
        except Exception as e:
            print(f"❌ Save failed: {e}")

    def batch_process(self, files: List[str], output_dir: str = "outputs") -> List[Dict[str, Any]]:
        Path(output_dir).mkdir(exist_ok=True)
        all_results = []
        print(f"🔄 Processing {len(files)} files...")
        for i, file in enumerate(files, 1):
            print(f"\n[{i}/{len(files)}] {Path(file).name}")
            result = self.extract_from_file(file)
            
            if result and "data" in result:
                all_results.append(result)
                self.save_results(result, str(Path(output_dir) / f"{Path(file).stem}_extracted.json"))
                
            # if i> 10:
            #     break
        self.save_results({"extractions": all_results}, str(Path(output_dir) / "all_extractions.json"))
        return all_results
    

    def list_excel_files(self,source_dir: str, extensions=("*.xlsx", "*.xlsm")) -> list[str]:
        """Return list of Excel files from a directory"""
        p = Path(source_dir)
        files = []
        for ext in extensions:
            files.extend(p.rglob(ext))  # rglob → recursive search
        return [str(f) for f in files]

    # Example usage
if __name__ == "__main__":
    # Single file processing
    source_dir = "Sevensix_dropbox/機密レベル2/営業本部/営業活動/営業報告書"
    extractor = ExcelDataExtractor()
    file_list = extractor.list_excel_files(source_dir)
    # results = extractor.batch_process(files=["20250602_blueqat_港様_v_清水 (4) (1).xlsx"])
    results = extractor.batch_process(files=file_list)

🔄 Processing 555 files...

[1/555] 20250701_NTTイノベーティブデバイス_守屋様_V_上瀬（20250623）.xlsm
Processing: 20250701_NTTイノベーティブデバイス_守屋様_V_上瀬（20250623）.xlsm (報告書, 97x21)
97
💾 Saved: outputs/20250701_NTTイノベーティブデバイス_守屋様_V_上瀬（20250623）_extracted.json

[2/555] 20250623_クアンテックー上瀧英郎_V_山下真弘（追記）.xlsm
Processing: 20250623_クアンテックー上瀧英郎_V_山下真弘（追記）.xlsm (営業 (フォーマット案_営業本部）, 71x21)
71
💾 Saved: outputs/20250623_クアンテックー上瀧英郎_V_山下真弘（追記）_extracted.json

[3/555] 20250606_名古屋大学_久野先生_V_河合.xlsm
Processing: 20250606_名古屋大学_久野先生_V_河合.xlsm (営業 (フォーマット案_営業本部）, 71x21)
71
💾 Saved: outputs/20250606_名古屋大学_久野先生_V_河合_extracted.json

[4/555] 20250722_富士通_蘇武洋平_R_大久保.xlsm
Processing: 20250722_富士通_蘇武洋平_R_大久保.xlsm (報告書, 97x21)
97
💾 Saved: outputs/20250722_富士通_蘇武洋平_R_大久保_extracted.json

[5/555] 20250718_NICT_北村光雄_V_清水卓也.xlsm
Processing: 20250718_NICT_北村光雄_V_清水卓也.xlsm (報告書, 97x21)
97
💾 Saved: outputs/20250718_NICT_北村光雄_V_清水卓也_extracted.json

[6/555] 20250707_長岡技術科学大学_岩橋様_V_久保寺（20250623）.xlsm
Processing: 20250707_長岡技術科学大学_岩橋様_V_久保寺（20250623）

/Users/jbc/Documents/SevenSix/ETL_pipeline/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


💾 Saved: outputs/20250605_フォトニック・エッジ_杉山様_V_河合_extracted.json

[30/555] 20250620_産総研_黒須_V_安野.xlsm
Processing: 20250620_産総研_黒須_V_安野.xlsm (営業 (フォーマット案_営業本部）, 71x21)
71
💾 Saved: outputs/20250620_産総研_黒須_V_安野_extracted.json

[31/555] 20250711_東京大学_北折曉 先生_V_津江.xlsm
Processing: 20250711_東京大学_北折曉 先生_V_津江.xlsm (報告書, 97x21)
97
💾 Saved: outputs/20250711_東京大学_北折曉 先生_V_津江_extracted.json

[32/555] 20250729_東京大学_井手口様_V確度進捗_宮田.xlsm
Processing: 20250729_東京大学_井手口様_V確度進捗_宮田.xlsm (報告書, 97x21)
97
💾 Saved: outputs/20250729_東京大学_井手口様_V確度進捗_宮田_extracted.json

[33/555] 20250627_NTTイノベーティブデバイス_坂本様_V_上瀬（20250623）.xlsm
Processing: 20250627_NTTイノベーティブデバイス_坂本様_V_上瀬（20250623）.xlsm (報告書, 97x21)
97
💾 Saved: outputs/20250627_NTTイノベーティブデバイス_坂本様_V_上瀬（20250623）_extracted.json

[34/555] 20250703_レーザーシステム_駒形方彦_V_山下真弘.xlsm
Processing: 20250703_レーザーシステム_駒形方彦_V_山下真弘.xlsm (報告書, 97x21)
97
💾 Saved: outputs/20250703_レーザーシステム_駒形方彦_V_山下真弘_extracted.json

[35/555] 20250613_岡本オプティクス_三上拓哉_R_安野.xlsm
Processing: 20250613_岡本オプティクス_三上拓哉_R_安

In [5]:
import pandas as pd
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
df = pd.read_excel(file_path, sheet_name="18期売上台帳", header=None)
key_column="受注番号"
# Step 2: Detect header rows by searching for key column name
header_indices = df.index[df.apply(lambda row: row.astype(str).str.contains(key_column).any(), axis=1)].tolist()
header_indices

[9, 318, 784, 1198, 1714, 1968, 2280, 2588, 2885, 3190, 3443, 3793, 4096]

In [24]:
import pandas as pd

def read_multi_table_excel(file_path, sheet_name="18期売上台帳", key_column="受注番号"):
    # Step 1: Read everything without headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    print(df.shape)
    df = df.dropna(how="all").dropna(axis=1, how='all') 
    print(df.shape)
    # Step 2: Detect header rows by searching for key column name
    header_indices = df.index[df.apply(lambda row: row.astype(str).str.contains(key_column).any(), axis=1)].tolist()
    
    all_chunks = []

    for i, start_idx in enumerate(header_indices):
        end_idx = header_indices[i+1] if i+1 < len(header_indices) else len(df)

        # extract block
        block = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # first row is the header
        header = block.iloc[0].fillna("").astype(str).tolist()

        # Make headers unique (important: since "in charge" and others repeat)
        seen = {}
        unique_headers = []
        for col in header:
            if col in seen:
                seen[col] += 1
                unique_headers.append(f"{col}_{seen[col]}")
            else:
                seen[col] = 0
                unique_headers.append(col)

        block.columns = unique_headers
        block = block.drop(0).reset_index(drop=True)

        # skip empty chunks
        if not block.empty:
            block["__TableBlock__"] = i - 1  # keep track of which block (month/section)
            all_chunks.append(block)

    # Step 3: Combine everything
    if not all_chunks:
        raise ValueError("No tables were found. Check key_column name.")

    final_df = pd.concat(all_chunks, ignore_index=True)
    return final_df


# Usage
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
clean_df = read_multi_table_excel(file_path,sheet_name="18期売上台帳", key_column="受注番号")

print(clean_df.head())
print(clean_df.shape)


Raw shape: (351105, 53)
Trimmed shape: (4334, 53)
Common columns: {'', '_3', '_4', '_2', '_1'}
        _3   _4   _2    _1  __TableBlock__
0  04  NaN  NaN  NaN  17.0               1
1  04  NaN  NaN    〇  17.0               1
2  04  NaN  NaN    〇  17.0               1
3  04  NaN  NaN    〇  17.0               1
4  04  NaN  NaN    〇  17.0               1
Final shape: (4312, 6)


In [28]:
import pandas as pd

def read_multi_table_excel(file_path, sheet_name="18期売上台帳", key_column="受注番号"):
    # Step 1: Read raw sheet without headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    df = df.dropna(how="all").dropna(axis=1, how="all")
    print(df.shape)
    # Step 2: Detect header rows by searching for key_column
    header_indices = df.index[
        df.apply(lambda row: row.astype(str).str.contains(key_column).any(), axis=1)
    ].tolist()

    tables = {}

    for i, start_idx in enumerate(header_indices):
        end_idx = header_indices[i+1] if i+1 < len(header_indices) else len(df)

        # extract one block
        block = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # --- Common heading (from first row) ---
        header = block.iloc[0].fillna("").astype(str).tolist()

        # Ensure unique column names
        seen = {}
        unique_headers = []
        for col in header:
            if col in seen:
                seen[col] += 1
                unique_headers.append(f"{col}_{seen[col]}")
            else:
                seen[col] = 0
                unique_headers.append(col)

        # Set headers
        block.columns = unique_headers
        block = block.drop(0).reset_index(drop=True)
        print(block.shape)
       
       
            
        if not block.empty:
            table_heading = f"Table_{i+1}"  # you can also take heading text from another row if needed
            block["__TableBlock__"] = i - 1

    if not tables:
        raise ValueError("No tables were found. Check key_column name.")

    return tables


# Usage
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
tables_dict = read_multi_table_excel(file_path, sheet_name="18期売上台帳", key_column="担当")

# Print each table separately
for heading, table in tables_dict.items():
    print(f"\n=== {heading} ===")
    print(table.head())
    print(table.shape)


Original shape: (4334, 53)
Table_1 shape: (298, 54)


ValueError: '受注番号' is not in list

In [34]:
import pandas as pd

def read_multi_table_excel(file_path, sheet_name="18期売上台帳", key_column="受注番号"):
    # Step 1: Read raw sheet without headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
    df = df.dropna(how="all").dropna(axis=1, how="all")
    print("Original shape:", df.shape)

    # Step 2: Detect header rows by searching for key_column
    header_indices = df.index[
        df.apply(lambda row: row.astype(str).str.contains(key_column).any(), axis=1)
    ].tolist()
    print(header_indices)
    tables = {}

    for i, start_idx in enumerate(header_indices):
        header_row = df.iloc[start_idx].astype(str).tolist()
        print("header_row: ",header_row)
        # --- Find key_column index safely ---
        key_col_idx = None
        for idx, val in enumerate(header_row):
            if key_column in val:
                key_col_idx = idx
                break
        if key_col_idx is None:
            raise ValueError(f"Key column '{key_column}' not found in header row {start_idx}")

        # --- Determine end of table by checking if key_column cell is empty ---
        end_idx = start_idx + 1
        while end_idx < len(df):
            key_cell = df.iloc[end_idx, key_col_idx]
            if pd.isna(key_cell) or str(key_cell).strip() == "":
                break
            end_idx += 1

        # extract one block
        block = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # --- Common heading (from first row) ---
        header = block.iloc[0].fillna("").astype(str).tolist()

        # Ensure unique column names
        seen = {}
        unique_headers = []
        for col in header:
            if col in seen:
                seen[col] += 1
                unique_headers.append(f"{col}_{seen[col]}")
            else:
                seen[col] = 0
                unique_headers.append(col)

        # Set headers
        block.columns = unique_headers
        block = block.drop(0).reset_index(drop=True)

        # Only keep block if it has data
        if not block.empty:
            table_heading = f"Table_{i+1}"  # Or take heading from row above
            block["__TableBlock__"] = i
            tables[table_heading] = block
            print(f"{table_heading} shape:", block.shape)

    if not tables:
        raise ValueError("No tables were found. Check key_column name.")

    return tables


# Usage
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
tables_dict = read_multi_table_excel(file_path, sheet_name="18期売上台帳", key_column="担当")

# Print each table separately
for heading, table in tables_dict.items():
    print(f"\n=== {heading} ===")
    print(table.head())
    print(table.shape)


Original shape: (4334, 53)
[9, 318, 784, 1198, 1714, 1968, 2280, 2588, 2885, 3190, 3443, 3793, 4096]
header_row:  ['完了', 'nan', 'nan', '担当', '受注番号', '受注日', '得意先名', '担当', '#', '型番', '商品名', '数量', '単位', '通貨単位', '販売単価', '送料/手数料', '小計', '円価小計', '円貨合計', '仕入先名', '通貨単位', '仕入単価', '送料/手数料', '外貨小計', '円価小計', '円価合計', '粗利益', '利益率', '備考', '売上日', '新規/既存', '区別', '営業所', 'アプリケーション', '受注経路', '量産', '入札', '他部署', '名称', '自社製品', '～2024/11納品', '2024/12～納品', 'nan', 'nan', 'ﾊﾞｲﾔｰ希望納期', 'メーカ回答 (一次)', 'メーカ回答 (二次)', '備考(納期)', '希望納期から遅延の理由', '売上予定月', '備考(作業用)', 'nan', '営業サイド自由記載']
Table_1 shape: (298, 54)
header_row:  ['済', 'nan', '12.0', 'N', 'O2312004N', '2023-12-04 00:00:00', '株式会社Preferred Networks', '金井', '1', 'TR-FC85S-N00', '100GBASE-SR4 QSFP28 Optical Transceiver', '6', 'PCS', 'JPY', '11000', '0', '66000', '66000', '66000', '在庫 S2310-2 InnoLight', 'USD', '39', '0', '234', '36504', '36504', '29496', '0.446909090909091', 'nan', '45268', '既存', '民間', '東京', 'DC', '営業案内', '-', '-', '-', 'Preferred Networks', 'nan',

ValueError: Key column '担当' not found in header row 318

In [56]:
import pandas as pd

def read_multi_table_excel(file_path, sheet_name="18期売上台帳", key_column="受注番号"):
    # Step 1: Read raw sheet without headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

    # Drop empty rows/columns first to speed up
    df = df.dropna(how="all").dropna(axis=1, how="all")
    print("Cleaned shape:", df.shape)

    tables = {}
    i = 0
    row_idx = 0

    while row_idx < len(df):
        row = df.iloc[row_idx]
        # Detect header row by checking if key_column exists in this row
        if row.astype(str).str.contains(key_column).any():
            header_row = row.astype(str).tolist()

            # Find key column index safely
            key_col_idx = None
            for idx, val in enumerate(header_row):
                if key_column in val:
                    key_col_idx = idx
                    break
            if key_col_idx is None:
                row_idx += 1
                continue  # should not happen

            # Determine end of table by checking key column empty
            end_idx = row_idx + 1
            while end_idx < len(df):
                key_cell = df.iloc[end_idx, key_col_idx]
                if pd.isna(key_cell) or str(key_cell).strip() == "":
                    break
                end_idx += 1

            # Extract block
            block = df.iloc[row_idx:end_idx].reset_index(drop=True)

            # --- Set headers ---
            header = block.iloc[0].fillna("").astype(str).tolist()
            seen = {}
            unique_headers = []
            for col in header:
                if col in seen:
                    seen[col] += 1
                    unique_headers.append(f"{col}_{seen[col]}")
                else:
                    seen[col] = 0
                    unique_headers.append(col)

            block.columns = unique_headers
            block = block.drop(0).reset_index(drop=True)

            # Only keep block if it has data
            if not block.empty:
                table_heading = f"Table_{i+1}"
                block["__TableBlock__"] = i
                tables[table_heading] = block
                print(f"{table_heading} shape:", block.shape)
                i += 1

            row_idx = end_idx
        else:
            row_idx += 1

    if not tables:
        raise ValueError("No tables were found. Check key_column name.")
    
   
    full_table = pd.concat(tables.values(), ignore_index=True)
    if "" in full_table.columns:
        full_table.drop(columns=[""],inplace=True)


    return full_table


# Usage
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
tables_dict = read_multi_table_excel(file_path, sheet_name="18期売上台帳", key_column="担当")
# --- Concatenate all tables into one ---


# print("Concatenated table shape:", all_tables_df.shape)
# print(all_tables_df.head())
# # Print each table separately
# for heading, table in tables_dict.items():
#     print(f"\n=== {heading} ===")
#     print(table.head())
#     print(table.columns)
    # print(table.shape)
    # table.to_csv(f"{heading}.csv",index=False)


Cleaned shape: (4334, 53)
Table_1 shape: (298, 54)
Table_2 shape: (455, 54)
Table_3 shape: (403, 54)
Table_4 shape: (505, 54)
Table_5 shape: (243, 54)
Table_6 shape: (301, 54)
Table_7 shape: (297, 54)
Table_8 shape: (286, 54)
Table_9 shape: (295, 54)
Table_10 shape: (242, 54)
Table_11 shape: (339, 54)
Table_12 shape: (292, 54)
Table_13 shape: (292, 54)


In [57]:
tables_dict.columns


Index(['完了', '_1', '担当', '受注番号', '受注日', '得意先名', '担当_1', '#', '型番', '商品名', '数量',
       '単位', '通貨単位', '販売単価', '送料/手数料', '小計', '円価小計', '円貨合計', '仕入先名', '通貨単位_1',
       '仕入単価', '送料/手数料_1', '外貨小計', '円価小計_1', '円価合計', '粗利益', '利益率', '備考', '売上日',
       '新規/既存', '区別', '営業所', 'アプリケーション', '受注経路', '量産', '入札', '他部署', '名称',
       '自社製品', '～2024/11納品', '2024/12～納品', '_2', '_3', 'ﾊﾞｲﾔｰ希望納期',
       'メーカ回答 (一次)', 'メーカ回答 (二次)', '備考(納期)', '希望納期から遅延の理由', '売上予定月', '備考(作業用)',
       '_4', '営業サイド自由記載', '__TableBlock__', '～2024/03納品', '2024/04以降納品', '_5',
       '_6', '_7', '客先注文書番号', '_8'],
      dtype='object')

In [43]:
# --- Concatenate all tables into one ---
all_tables_df = pd.concat(tables_dict.values(), ignore_index=True)

print("Concatenated table shape:", all_tables_df.shape)
print(all_tables_df.head())

Concatenated table shape: (4248, 61)
  完了        _1 担当        受注番号                  受注日              得意先名 担当_1  #  \
0  済  04  17.0  M  O2304026M.  2023-04-03 00:00:00           株式会社ニコン   岩田  1   
1  済  04  17.0  D   O2304043D  2023-04-03 00:00:00          株式会社ディスコ   村上  1   
2  済  04  17.0  F   O2304049F  2023-04-03 00:00:00  株式会社トーメーコーポレーション   西尾  1   
3  済  04  17.0  F   O2304049F  2023-04-03 00:00:00  株式会社トーメーコーポレーション   西尾  1   
4  済  04  17.0  D   O2304055D  2023-04-03 00:00:00          株式会社ディスコ   村上  1   

                                                  型番  ...   _4 営業サイド自由記載  \
0  OEOFF-1986.5-0.05-3-0.05-99.9-2-PM-GDF-10/130-...  ...  NaN       NaN   
1                 EYP-DFB-1064-00040-1500-BFY02-0006  ...  NaN       NaN   
2                                 FCM-06-9-J0-2-L-RS  ...  NaN       NaN   
3                                 FCM-06-9-J0-2-L-RS  ...  NaN       NaN   
4                 EYP-DFB-1064-00040-1500-BFY02-0006  ...  NaN       NaN   

  __TableBlock__ ～2024/03

In [44]:
all_tables_df.shape

(4248, 61)

In [75]:
import pandas as pd
import glob
import os

def combine_excel_files(folder_path: str, sheet_name: str = 0, skip_files: list = []) -> pd.DataFrame:
    # Find all Excel files in the folder
    files = glob.glob(f"{folder_path}/*.xlsx")
    
    df_list = []
    for file in files:
        file_name = os.path.basename(file)  # only file name
        if file_name in skip_files:         # compare names only
            print(f"Skipping {file_name}")
            continue
        try:
            df = pd.read_excel(file, sheet_name=sheet_name)
            df = df.dropna(how="all").dropna(axis=1, how="all")
            
            df_list.append(df)
            print("file: ",df.shape)
            print(f"{file_name} processed")
        except Exception as e:
            print(f"Error reading {file_name}: {e}")
    
    if not df_list:
        print("⚠️ No valid DataFrames to concatenate.")
        return pd.DataFrame()  # return empty DataFrame
    
    # Combine into one DataFrame
    full_table = pd.concat(df_list, ignore_index=True)
    print("columns: ",full_table.columns)
    if " " in full_table.columns or '　' in full_table.columns or "" in full_table.columns:
        full_table.drop(columns=['　'],inplace=True)
    return full_table
# Example usage:
df = combine_excel_files(
    "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳", 
    
    # sheet_name="SalesData",
    skip_files=["18期_売上_納期管理台帳.xlsx"]
)
print(df.shape)
df.columns


file:  (3989, 76)
【使用不可】★14-15期_納期.xlsx processed
file:  (3562, 29)
【使用不可】16期-17期_納期 (1).xlsx processed
file:  (1928, 36)
【使用不可】17期_納期(20230401~20231130) (1).xlsx processed
Skipping 18期_売上_納期管理台帳.xlsx
columns:  Index(['　', '契約No', 'P.O.№', '客先名', 'アイテム＃', '商品コード', '商品名', '数量', '出荷数',
       '注文書番号',
       ...
       'メーカ回答\n9/14', 'メーカ回答\n9/22', 'メーカ回答\n9/29', 'メーカ回答\n10/13',
       'メーカ回答\n10/20', 'メーカ回答\n10/27', 'メーカ回答\n11/2', 'メーカ回答\n11/2.1',
       'メーカ回答\n11/10', 'メーカ回答\n11/24'],
      dtype='object', length=101)
(9479, 100)


Index(['契約No', 'P.O.№', '客先名', 'アイテム＃', '商品コード', '商品名', '数量', '出荷数', '注文書番号',
       '仕入先コード', '仕入先名', 'ﾒｰｶｰｺｰﾄﾞ', 'ﾒｰｶｰ名', 'Mへの発注金額', '受注日', 'ﾊﾞｲﾔｰ希望納期',
       'メーカ回答\n初回', 'メーカ回答\n2/22', 'メーカ回答\n3/19', 'メーカ回答\n3/25', 'メーカ回答\n4/1',
       'メーカ回答\n4/8', 'メーカ回答\n4/15', 'メーカ回答\n4/21', 'メーカ回答\n4/28', 'メーカ回答\n5/7',
       'メーカ回答\n5/13', 'メーカ回答\n5/20', 'メーカ回答\n5/27', 'メーカ回答\n6/3',
       'メーカ回答\n6/10', 'メーカ回答\n6/17', 'メーカ回答\n6/24', 'メーカ回答\n7/1', 'メーカ回答\n7/9',
       'メーカ回答\n7/16', 'メーカ回答\n7/26', 'メーカ回答\n7/30', 'メーカ回答\n8/6',
       'メーカ回答\n8/13', 'メーカ回答\n8/19', 'メーカ回答\n8/26', 'メーカ回答\n9/1', 'メーカ回答\n9/8',
       'メーカ回答\n9/17', 'メーカ回答\n9/24（BWTは9/22）', 'メーカ回答\n9/30（Optizoneは9/27）',
       'メーカ回答\n10/7', 'メーカ回答\n10/15', 'メーカ回答\n10/21', 'メーカ回答\n10/2８',
       'メーカ回答\n11/4', 'メーカ回答\n11/12　（BWTは11/10回答時）', 'メーカ回答\n11/18',
       'メーカ回答\n11/25', 'メーカ回答\n12/2', 'メーカ回答\n12/10', 'メーカ回答\n12/16',
       'メーカ回答\n12/24', 'メーカ回答\n1/6', 'メーカ回答\n1/14', 'メーカ回答\n1/21',
       'メーカ回答\n1/28(BWTは1/24）', 'メーカ回答\n2

In [65]:
df = df.dropna(how='all')   # Drop fully empty rows
df = df.dropna(axis=1, how='all')  # Drop fully empty columns
df.shape

(9479, 101)

In [66]:
df.head(-10)

,,契約No,P.O.№,客先名,アイテム＃,商品コード,商品名,数量,出荷数,注文書番号,...,メーカ回答\n9/14,メーカ回答\n9/22,メーカ回答\n9/29,メーカ回答\n10/13,メーカ回答\n10/20,メーカ回答\n10/27,メーカ回答\n11/2,メーカ回答\n11/2.1,メーカ回答\n11/10,メーカ回答\n11/24
0,済,O2012147K,契約番号：20-10-21-00158555,国立研究開発法人 理化学研究所,3.0,Yb-doped PCF amplifier including,• Forward-pumping monolithic architecture_x000...,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,済,O2104002N,I210000185-0,ミハル通信株式会社,1.0,MPC-(2+1)x1-B-915/10W-1550-110/454-000-3x50-1.0m,Multi-mode Pump Combiner((2+1)x1),30,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,済,O2104003G,AG0Q50N0A,三菱電機株式会社,1.0,PMFC-59-1-50-F-NNN-BBB-P-1.1,"Polarization Maintaining Filter Coupler, 1590+...",6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,済,O2104004K,2021/01/19付発注書,東京工業大学,2.0,A911-100-201,USB-RS485 Adapter kit for new BASIK generation,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,済,O2104004K,2021/01/19付発注書,東京工業大学,1.0,K112,Koheras BASIK C15 PM FM 1560.48nm to 2 s.f. re...,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9464,NaN,O2311112M,hac-231127-00456,270000,1.0,MY-1375,100g Plastic Bottle (IR=1.379),1,0,O2311112M-0026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9465,NaN,O2311113E,W2301122004,438400,1.0,WDM-1-520770-2-L-0.25-2 (SR24514),シングルモードWDMカプラ 1X2型520 & 770nm、FC/APCコネクタ付き、900...,8,0,O2311113E-0049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9466,NaN,O2311113E,W2301122004,84000,2.0,WDM-1-9851560-2-L-0.25-F,シングルモードWDMカプラ 1X2型 980 & 1560nm、FC/APCコネクタ付き、9...,8,0,O2311113E-0049,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9467,NaN,O2311114W,OR-00013773<R05>,220500,1.0,K976AAHRN-27.00W,976nm 27W Wavelength-Stabilized Fiber Coupled ...,1,0,O2311114W-0009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
import pandas as pd

def read_multi_table_excel(file_path, sheet_name=0, key_column="受注番号"):
    # Step 1: Read everything without headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

    # Step 2: Detect header rows by searching for key column name
    header_indices = df.index[df.apply(lambda row: row.astype(str).str.contains(key_column).any(), axis=1)].tolist()

    all_chunks = []
    skip_markers = ["受注額合計", "仕入額合計", "粗利益", "利益率"]  # unwanted summary rows

    for i, start_idx in enumerate(header_indices):
        end_idx = header_indices[i+1] if i+1 < len(header_indices) else len(df)

        # extract block
        block = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # first row is the header
        header = block.iloc[0].fillna("").astype(str).tolist()
        print(header)

        # Make headers unique
        seen = {}
        unique_headers = []
        for col in header:
            if col in seen:
                seen[col] += 1
                unique_headers.append(f"{col}_{seen[col]}")
            else:
                seen[col] = 0
                unique_headers.append(col)

        block.columns = unique_headers
        block = block.drop(0).reset_index(drop=True)

        # --- Filter unwanted rows ---
        block = block[
            ~block.astype(str).apply(lambda row: any(marker in "".join(row) for marker in skip_markers), axis=1)
        ]

        # skip empty chunks
        if not block.empty:
            block["__TableBlock__"] = i + 1  # track which block/month
            all_chunks.append(block)

    # Step 3: Combine everything
    if not all_chunks:
        raise ValueError("No tables were found. Check key_column name.")

    final_df = pd.concat(all_chunks, ignore_index=True)

    return final_df


# Usage
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
clean_df = read_multi_table_excel(file_path,sheet_name="18期売上台帳", key_column="受注番号")

print(clean_df.head())
print(clean_df.tail())


  完了        _1 担当        受注番号                  受注日              得意先名 担当_1  #  \
0  済  04  17.0  M  O2304026M.  2023-04-03 00:00:00           株式会社ニコン   岩田  1   
1  済  04  17.0  D   O2304043D  2023-04-03 00:00:00          株式会社ディスコ   村上  1   
2  済  04  17.0  F   O2304049F  2023-04-03 00:00:00  株式会社トーメーコーポレーション   西尾  1   
3  済  04  17.0  F   O2304049F  2023-04-03 00:00:00  株式会社トーメーコーポレーション   西尾  1   
4  済  04  17.0  D   O2304055D  2023-04-03 00:00:00          株式会社ディスコ   村上  1   

                                                  型番  ...   _4 営業サイド自由記載  \
0  OEOFF-1986.5-0.05-3-0.05-99.9-2-PM-GDF-10/130-...  ...  NaN       NaN   
1                 EYP-DFB-1064-00040-1500-BFY02-0006  ...  NaN       NaN   
2                                 FCM-06-9-J0-2-L-RS  ...  NaN       NaN   
3                                 FCM-06-9-J0-2-L-RS  ...  NaN       NaN   
4                 EYP-DFB-1064-00040-1500-BFY02-0006  ...  NaN       NaN   

  __TableBlock__ ～2024/03納品 2024/04以降納品   _5   _6   _7 客先注文書番号

In [3]:
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
sheet_name="18期売上台帳"
key_column="受注番号"
# Step 1: Read everything without headers
df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

# Step 2: Detect header rows by searching for key column name
header_indices = df.index[df.apply(lambda row: row.astype(str).str.contains(key_column).any(), axis=1)].tolist()
header_indices

[9, 318, 784, 1198, 1714, 1968, 2280, 2588, 2885, 3190, 3443, 3793, 4096]

In [4]:

all_chunks = []
skip_markers = ["受注額合計", "仕入額合計", "粗利益", "利益率"]  # unwanted summary rows

for i, start_idx in enumerate(header_indices):
    end_idx = header_indices[i+1] if i+1 < len(header_indices) else len(df)

    # extract block
    block = df.iloc[start_idx:end_idx].reset_index(drop=True)
    
    # first row is the header
    header = block.iloc[0].fillna("").astype(str).tolist()
    print(header)

['完了', '', '', '担当', '受注番号', '受注日', '得意先名', '担当', '#', '型番', '商品名', '数量', '単位', '通貨単位', '販売単価', '送料/手数料', '小計', '円価小計', '円貨合計', '仕入先名', '通貨単位', '仕入単価', '送料/手数料', '外貨小計', '円価小計', '円価合計', '粗利益', '利益率', '備考', '売上日', '新規/既存', '区別', '営業所', 'アプリケーション', '受注経路', '量産', '入札', '他部署', '名称', '自社製品', '～2024/11納品', '2024/12～納品', '', '', 'ﾊﾞｲﾔｰ希望納期', 'メーカ回答 (一次)', 'メーカ回答 (二次)', '備考(納期)', '希望納期から遅延の理由', '売上予定月', '備考(作業用)', '', '営業サイド自由記載']
['完了', '', '', '担当', '受注番号', '受注日', '得意先名', '担当', '#', '型番', '商品名', '数量', '単位', '通貨単位', '販売単価', '送料/手数料', '小計', '円価小計', '円貨合計', '仕入先名', '通貨単位', '仕入単価', '送料/手数料', '外貨小計', '円価小計', '円価合計', '粗利益', '利益率', '備考', '売上日', '新規/既存', '区別', '営業所', 'アプリケーション', '受注経路', '量産', '入札', '他部署', '名称', '自社製品', '～2024/03納品', '2024/04以降納品', '', '', 'ﾊﾞｲﾔｰ希望納期', 'メーカ回答 (一次)', 'メーカ回答 (二次)', '備考(納期)', '希望納期から遅延の理由', '売上予定月', '備考(作業用)', '', '']
['完了', '', '', '担当', '受注番号', '受注日', '得意先名', '担当', '#', '型番', '商品名', '数量', '単位', '通貨単位', '販売単価', '送料/手数料', '小計', '円価小計', '円貨合計', '仕入先名', '通貨単位', '仕入単価', '送

In [2]:

all_chunks = []
skip_markers = ["受注額合計", "仕入額合計", "粗利益", "利益率"]  # unwanted summary rows

for i, start_idx in enumerate(header_indices):
    end_idx = header_indices[i+1] if i+1 < len(header_indices) else len(df)

    # extract block
    block = df.iloc[start_idx:end_idx].reset_index(drop=True)

    # first row is the header
    header = block.iloc[0].fillna("").astype(str).tolist()
    print(header)

    # Make headers unique
    seen = {}
    unique_headers = []
    for col in header:
        if col in seen:
            seen[col] += 1
            unique_headers.append(f"{col}_{seen[col]}")
        else:
            seen[col] = 0
            unique_headers.append(col)

    block.columns = unique_headers
    block = block.drop(0).reset_index(drop=True)

    # --- Filter unwanted rows ---
    block = block[
        ~block.astype(str).apply(lambda row: any(marker in "".join(row) for marker in skip_markers), axis=1)
    ]

    # skip empty chunks
    if not block.empty:
        block["__TableBlock__"] = i + 1  # track which block/month
        all_chunks.append(block)

# Step 3: Combine everything
if not all_chunks:
    raise ValueError("No tables were found. Check key_column name.")

final_df = pd.concat(all_chunks, ignore_index=True)

# return final_df

,完了,,_1,担当,受注番号,受注日,得意先名,担当_1,#,型番,...,_4,営業サイド自由記載,__TableBlock__,～2024/03納品,2024/04以降納品,_5,_6,_7,客先注文書番号,_8
0,済,04,17.0,M,O2304026M.,2023-04-03 00:00:00,株式会社ニコン,岩田,1,OEOFF-1986.5-0.05-3-0.05-99.9-2-PM-GDF-10/130-...,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,済,04,17.0,D,O2304043D,2023-04-03 00:00:00,株式会社ディスコ,村上,1,EYP-DFB-1064-00040-1500-BFY02-0006,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,済,04,17.0,F,O2304049F,2023-04-03 00:00:00,株式会社トーメーコーポレーション,西尾,1,FCM-06-9-J0-2-L-RS,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,済,04,17.0,F,O2304049F,2023-04-03 00:00:00,株式会社トーメーコーポレーション,西尾,1,FCM-06-9-J0-2-L-RS,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,済,04,17.0,D,O2304055D,2023-04-03 00:00:00,株式会社ディスコ,村上,1,EYP-DFB-1064-00040-1500-BFY02-0006,...,NaN,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd

def read_multi_table_excel(file_path, sheet_name=0, key_column="受注番号"):
    # Step 1: Read everything without headers
    df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)

    # Step 2: Detect header rows
    header_indices = df.index[
        df.apply(lambda row: row.astype(str).str.contains(key_column).any(), axis=1)
    ].tolist()

    all_chunks = []
    skip_markers = ["受注額合計", "仕入額合計", "粗利益", "利益率", "済"]

    for i, start_idx in enumerate(header_indices):
        end_idx = header_indices[i+1] if i+1 < len(header_indices) else len(df)

        # --- Extract block ---
        block = df.iloc[start_idx:end_idx].reset_index(drop=True)

        # --- First row is header ---
        header = block.iloc[0].fillna("").astype(str).tolist()

        # Make headers unique
        seen = {}
        unique_headers = []
        for col in header:
            if col in seen:
                seen[col] += 1
                unique_headers.append(f"{col}_{seen[col]}")
            else:
                seen[col] = 0
                unique_headers.append(col)

        block.columns = unique_headers
        block = block.drop(0).reset_index(drop=True)

        # --- Find stop point (summary/junk rows) ---
        stop_idx = None
        for j, row in block.iterrows():
            row_text = "".join(row.astype(str))
            if any(marker in row_text for marker in skip_markers):
                stop_idx = j
                break
        if stop_idx:
            block = block.iloc[:stop_idx]

        # --- Keep only non-empty block ---
        if not block.empty:
            block["__TableBlock__"] = i + 1
            all_chunks.append(block)

    if not all_chunks:
        raise ValueError("No valid tables found. Check key_column name.")

    final_df = pd.concat(all_chunks, ignore_index=True)
    return final_df
# Usage
file_path = "Sevensix_dropbox/機密レベル3/企画管理本部/業務推進部/売上台帳/18期_売上_納期管理台帳.xlsx"
clean_df = read_multi_table_excel(file_path,sheet_name="18期売上台帳", key_column="受注番号")


In [14]:
clean_df.head()

,完了,,_1,担当,受注番号,受注日,得意先名,担当_1,#,型番,...,ﾊﾞｲﾔｰ希望納期,メーカ回答 (一次),メーカ回答 (二次),備考(納期),希望納期から遅延の理由,売上予定月,備考(作業用),_6,_7,__TableBlock__
0,NaN,NaN,9.0,E,O2409001E,45537,国立大学法人徳島大学,,1,EB-FH52,...,NaN,NaN,NaN,NaN,NaN,1,O2408131E-2へ,NaN,NaN,11


In [15]:
clean_df.shape

(1, 54)

In [8]:
block = block.dropna(how="all")

print(df.shape)
df = df.dropna(axis=1, how='all')  # Drop fully empty columns
print(df.shape)

(350735, 61)
(350735, 60)
